# Gaussian pulse evokes travelling waves in 1D SNN

- https://www.nature.com/articles/ncomms4675#Sec24

- https://www.nature.com/articles/ncomms4675.pdf

- https://static-content.springer.com/esm/art%3A10.1038%2Fncomms4675/MediaObjects/41467_2014_BFncomms4675_MOESM1276_ESM.pdf

In [1]:
# fileName

fileName = 'eg_1D_evocked_waves' 

In [2]:
# set libs

import pyNN.spiNNaker as sim
import numpy as np
import matplotlib.pyplot as plt

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation setting

dt         = 1          # (ms) #0.1
simtime    = 200       # (ms)
sim.setup(timestep = dt, 
          min_delay = 1,
          max_delay = 14) 


2021-04-11 10:38:59 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-11 10:38:59 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-11 10:38:59 WARNING: /home/spinnaker/CNS-tutorial-2021/reports has 6 old reports that have not been closed
2021-04-11 10:38:59 WARNING: /home/spinnaker/CNS-tutorial-2021/application_generated_data_files has 7 old reports that have not been closed
2021-04-11 10:38:59 INFO: Setting time scale factor to 1.
2021-04-11 10:38:59 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

In [4]:
# making the network 
from pyNN import space 


popName = 'network'

pops = {}
n_cells = {}
n_cells[popName] = 1000

pops[popName] = sim.Population(n_cells[popName], # number of cells
                       sim.IF_cond_exp, # cell model
                       sim.IF_cond_exp.default_parameters, # cell parameters
                       structure = space.Line(dx=1.0, 
                                              x0=0.0, 
                                              y=0.0, 
                                              z=0.0),
                       label=popName)

pops[popName].record(['spikes', 'v', 'gsyn_exc', 'gsyn_inh'])



In [5]:

# select 750 excitatory cells and 250 inhibitory cells
import random

n_cells = {}
n_cells['network'] = 1000
n_cells['Exc'] = 750
n_cells['Inh'] = 250

idx_cells = {}
idx_cells['Tot'] = np.arange(0, n_cells['network'])
idx_cells['Exc'] = np.sort(random.sample(list(idx_cells['Tot']), n_cells['Exc']))
idx_cells['Inh'] = np.sort(list(set(idx_cells['Tot']) - set(idx_cells['Exc'])))

#for cell in idx_cells['Exc']:
    #print(cell)

def compute_manual_list(idx_cells_i, idx_cells_j, weights, d_thresh, p_thresh, width): 
        v_c = 0.35 # m/s
        d0 = 1 #ms
        scale = 1

        connections = {}
        probabilities = {}
        distances = {}

        connections = []
        probabilities = []
        distances = []
        for pre in idx_cells_i:
            for post in idx_cells_j:
                d_ij = np.sqrt((pre - post)**2)
                delay = d0 + d_ij / v_c
                if d_ij > d_thresh: 
                    distances.append(d_ij)
                    p_ij = scale*np.exp(-0.5 * (d_ij**2/width**2))
                    probabilities.append(p_ij)

                    if p_ij > p_thresh:
                        connections.append([pre, post, weights, delay])#, [d_ij, p_ij]])

        return connections, distances, probabilities
        
# computation step
weight = {('Exc', 'Tot') : 0.018,
          ('Inh', 'Tot') : 0.180,
          }

d_thresh = {('Exc', 'Tot') : 0.0,
          ('Inh', 'Tot') : 0.0,
          }


p_thresh = {('Exc', 'Tot') : 0.01,
          ('Inh', 'Tot') : 0.01,
          }


width = {('Exc', 'Tot') : 1,
          ('Inh', 'Tot') : 1,
          }


connections = {}
distances = {}
probabilities = {}

for i in ['Exc', 'Inh']:
    connections[i,'Tot'], distances[i,'Tot'], probabilities[i,'Tot'] = compute_manual_list(idx_cells[i], 
                                                                           idx_cells['Tot'], 
                                                                           weight[i, 'Tot'], 
                                                                           d_thresh[i, 'Tot'], 
                                                                           p_thresh[i, 'Tot'], 
                                                                           width[i, 'Tot'])

do_run=True
if do_run:
    # visual check, e.g. with Exc > Exc
    fig, ax = plt.subplots(1,2, figsize=(11,5))
    fig.tight_layout(pad=3)
    axes_list = fig.axes

    axes_list[0].plot(np.asarray(connections['Exc', 'Tot']).T[0], np.asarray(connections['Exc', 'Tot']).T[1],'go', label='Exc (i) - Tot (j)')
    axes_list[0].plot(np.asarray(connections['Inh', 'Tot']).T[0], np.asarray(connections['Inh', 'Tot']).T[1],'r+', label='Inh (i) - Tot (j)')
    axes_list[0].plot(np.unique(np.asarray(connections['Exc', 'Tot']).T[0]), idx_cells['Exc'], 'ko')
    axes_list[0].plot(np.unique(np.asarray(connections['Inh', 'Tot']).T[0]), idx_cells['Inh'], 'y+')

    axes_list[0].grid()
    axes_list[0].legend()
    axes_list[0].set_title('scatter plot of connections')
    axes_list[0].set_xlabel('i cells')
    axes_list[0].set_ylabel('j cells')
    axes_list[0].set_xlim(500,525)
    axes_list[0].set_ylim(500,525)

    axes_list[1].plot(distances['Exc', 'Tot'],probabilities['Exc', 'Tot'],'g+')
    axes_list[1].plot(distances['Inh', 'Tot'],probabilities['Inh', 'Tot'],'r+')

    axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Exc', 'Tot']**2)), 'g:')
    axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Inh', 'Tot']**2)), 'r:')

    axes_list[1].grid()
    axes_list[1].set_xlim(0,15)
    axes_list[1].set_ylim(0,1)
    axes_list[1].axhline(p_thresh['Inh', 'Tot'], color='k', label='p_threshold', )
    axes_list[1].set_title('probability as function of distance')
    axes_list[1].set_xlabel('distance')
    axes_list[1].set_ylabel('probability')
    axes_list[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
"""
%matplotlib
plt.plot(np.unique(np.asarray(connections['Exc', 'Tot']).T[0]), idx_cells['Exc'], '+')
plt.plot(np.unique(np.asarray(connections['Inh', 'Tot']).T[0]), idx_cells['Inh'], 'o')
plt.xlim(500,525)
plt.ylim(500,525)
len(np.unique(np.asarray(connections['Exc', 'Tot']).T[0])), len(np.unique(np.asarray(connections['Inh', 'Tot']).T[0]))
"""

#np.unique(np.asarray(connections['Inh', 'Tot']).T[0])

"\n%matplotlib\nplt.plot(np.unique(np.asarray(connections['Exc', 'Tot']).T[0]), idx_cells['Exc'], '+')\nplt.plot(np.unique(np.asarray(connections['Inh', 'Tot']).T[0]), idx_cells['Inh'], 'o')\nplt.xlim(500,525)\nplt.ylim(500,525)\nlen(np.unique(np.asarray(connections['Exc', 'Tot']).T[0])), len(np.unique(np.asarray(connections['Inh', 'Tot']).T[0]))\n"

In [7]:
# make projection
projs = {}

do_run = True
if do_run:
    receptors = {}
    receptors['Exc', 'Tot'] = 'excitatory'
    receptors['Inh', 'Tot'] = 'inhibitory'

    projs = {}
    for i in ['Exc', 'Inh']:
        projs[i,'network'] = sim.Projection(
                                pops['network'],
                                pops['network'],
                                connector = sim.FromListConnector(connections[i,'Tot']), 
                                receptor_type = receptors[i,'Tot'],
                                space = space.Space(axes = 'x'),            
                                label = i + ' - tot',
                            )
            

In [8]:
# make the thalamic gaussian pulse

popName = 'Thalamus'  
tot_cells = n_cells['network']

spike_times = [[]]*(tot_cells) #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(tot_cells*0.40, tot_cells*0.60) for i in range(tot_cells)]

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = [abs(np.random.normal(loc=50, scale=5)) for n in range(5)]


pops[popName] = sim.Population(tot_cells, 
                               sim.SpikeSourceArray(spike_times),
                               structure = space.Line(dx=1.0, 
                                              x0=0.0, 
                                              y=0.0, 
                                              z=0.0),
                               
                               label = popName)

pops[popName].record('spikes')

do_plot = True
if do_plot:
    fig, axes = plt.subplots(1,1)
    axes_list = fig.axes
    axes_list[0].eventplot(spike_times)
    axes_list[0].set_xlabel('[ms]')
    axes_list[0].set_ylabel('thalamic spike sources')
    axes_list[0].set_title('rasterplot')
    axes_list[0].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# make the thalamic projections

projs['Thalamus', 'Exc'] = sim.Projection(pops['Thalamus'], 
                                       pops['network'],
                                       connector = sim.OneToOneConnector(),
                                       synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                       receptor_type = 'excitatory',
                                       space = space.Space(axes = 'x'),                                                     
                                       label = 'Thalamus - ' + 'Exc')

In [10]:
# run simulation

sim.run(simtime) # simtime=500 [ms]

2021-04-11 10:39:14 INFO: Simulating for 200 1.0ms timesteps using a hardware timestep of 1000us
2021-04-11 10:39:14 INFO: Starting execution process
2021-04-11 10:39:18 INFO: Time 0:00:03.969220 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-11 10:39:29 INFO: Time 0:00:11.422348 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-11 10:39:35 INFO: Time 0:00:05.834519 taken by PartitionAndPlacePartitioner
Created spalloc job 5932935
2021-04-11 10:39:35 INFO: Created spalloc job 5932935
Waiting for board power commands to complete.
2021-04-11 10:39:35 INFO: Waiting for board power commands to complete.
2021-04-11 10:39:40 INFO: Time 0:00:05.048545 taken by SpallocAllocator
2021-04-11 1

200.0

In [11]:
# save the results
outputs = {}


outputs['network'] = pops['network'].get_data()
for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
    pops['network'].write_data(fileName + '_' + str(recording) + '.pkl')

Getting spikes for network
|0%                          50%                         100%|
Getting v for network
|0%                          50%                         100%|
Getting gsyn_exc for network
|0%                          50%                         100%|
Getting gsyn_inh for network
|0%                          50%                         100%|
Getting spikes for network
|0%                          50%                         100%|
Getting v for network
|0%                          50%                         100%|
Getting gsyn_exc for network
|0%                          50%                         100%|
Getting gsyn_inh for network
|0%                          50%                         100%|
Getting spikes for network
|0%                          50%                         100%|
Getting v for network
|0%                          50%                         100%|
Getting gsyn_exc for network
|0%                          50%                         100%|
Getting gsyn_in

In [12]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

In [13]:
# check results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh


dict_keys([('network', 'v'), ('network', 'gsyn_exc'), ('network', 'gsyn_inh'), ('network', 'spikes')])

In [14]:
# check the state variable

fig, axes = plt.subplots(1, 1, figsize=(5,9))
fig.tight_layout(pad=5)
fig.suptitle('rasterplot and voltage plot')
axes_list = fig.axes
idx = 0
value = 'network'
axes_list[idx].eventplot(results[value, 'spikes'], colors='r')
imv = axes_list[idx].imshow(results[value, 'v'].T)
axes_list[idx].set_title(str(value) + ' layer')
axes_list[idx].set_xlabel('[ms]')
axes_list[idx].set_ylabel('cells')
axes_list[idx].set_xlim(0, simtime)
fig.colorbar(imv, ax=axes_list[idx], fraction=0.020, label='[mV]')
    



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
#results['Exc', 'spikes']

In [16]:
# check the conductances

 
fig, axes = plt.subplots(1, 2)
fig.tight_layout(pad=5)
axes_list = fig.axes
fig.suptitle('rasterplot and gsyn plot of network')
layer = 'network'
for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):

    #axes_list[idx].eventplot(results[value, 'spikes'], colors='r')
    im = axes_list[idx].imshow(results[layer, gsyn].T)
    axes_list[idx].set_title(str(gsyn))
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('cells')
    axes_list[idx].set_xlim(0, simtime)
    fig.colorbar(im, ax=axes_list[idx], fraction=0.020, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
#.get(['source', 'target', 'weight', 'delay'], format='list')
projs

{('Exc', 'network'): projection Exc - tot,
 ('Inh', 'network'): projection Inh - tot,
 ('Thalamus', 'Exc'): projection Thalamus - Exc}

In [18]:
projs['Exc', 'network'].get(['source', 'target', 'weight', 'delay'], format='list')[0:100]

2021-04-11 10:40:16 WARNING: Ignoring request to get source as with_address=True. 
2021-04-11 10:40:16 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between network and network
|0%                          50%                         100%|


array([( 0,  1, 0.01800156,  4.), ( 0,  2, 0.01800156,  7.),
       ( 0,  3, 0.01800156, 10.), ( 1,  0, 0.01800156,  4.),
       ( 1,  2, 0.01800156,  4.), ( 1,  3, 0.01800156,  7.),
       ( 1,  4, 0.01800156, 10.), ( 2,  0, 0.01800156,  7.),
       ( 2,  1, 0.01800156,  4.), ( 2,  3, 0.01800156,  4.),
       ( 2,  4, 0.01800156,  7.), ( 2,  5, 0.01800156, 10.),
       ( 3,  0, 0.01800156, 10.), ( 3,  1, 0.01800156,  7.),
       ( 3,  2, 0.01800156,  4.), ( 3,  4, 0.01800156,  4.),
       ( 3,  5, 0.01800156,  7.), ( 3,  6, 0.01800156, 10.),
       ( 4,  1, 0.01800156, 10.), ( 4,  2, 0.01800156,  7.),
       ( 4,  3, 0.01800156,  4.), ( 4,  5, 0.01800156,  4.),
       ( 4,  6, 0.01800156,  7.), ( 4,  7, 0.01800156, 10.),
       ( 5,  2, 0.01800156, 10.), ( 5,  3, 0.01800156,  7.),
       ( 5,  4, 0.01800156,  4.), ( 5,  6, 0.01800156,  4.),
       ( 5,  7, 0.01800156,  7.), ( 5,  8, 0.01800156, 10.),
       ( 7,  4, 0.01800156, 10.), ( 7,  5, 0.01800156,  7.),
       ( 7,  6, 0.018001